In [1]:
import pandas as pd     
import warnings
import numpy as np
warnings.simplefilter(action='ignore', category=FutureWarning)
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default='notebook'



def normalize(df, how="min-max"):
    if how == "min-max" :
        return (df - df.min())/(df.max()-df.min())
    if how == "mean" :
        return (df-df.mean())/df.std()


def get_cp_indices_df(df_cp, df_indices, selected_coutry = None, data_resolution = "M", normalization="mean", normalize_cp=True, smoothing = False, smoothing_duration=6, indices=None) :


    if not selected_coutry : selected_coutry = df_cp.country.unique()

    df_cp_filter = df_cp[df_cp.country.isin(selected_coutry) ]
    df_cp_filter = df_cp_filter.groupby([df_cp_filter.index.to_period(data_resolution)]).mean().reset_index()
    df_cp_filter = df_cp_filter.set_index(df_cp_filter.timestamp).drop(["timestamp"], axis=1)


    if indices : df_indices = df_indices[indices]
    df_indices_filter = df_indices.groupby([df_indices.index.to_period(data_resolution)]).mean().reset_index()
    df_indices_filter = df_indices_filter.set_index(df_indices_filter.timestamp).drop(["timestamp"], axis=1)

    df_cp_indices = df_cp_filter.join(df_indices_filter)
    df_cp_indices = df_cp_indices[df_cp_indices.index.year>=1982]
    df_cp_indices = normalize(df_cp_indices,how=normalization)


    if not normalize_cp :  
        df_cp_indices.capacity_factor = df_cp_filter[df_cp_filter.index.year>=1982]


    if smoothing : 
        df_cp_indices = df_cp_indices.rolling(smoothing_duration, center=True).mean()


    return df_cp_indices

    

def compare_cp_index(df_cp_indices, smoothing = False, smoothing_duration=6, indices = None, start=1990, end=2019, show_plot = True):
    if indices : 
        df_cp_indices = df_cp_indices[indices+["capacity_factor"]] 
    df_cp_indices = df_cp_indices[(df_cp_indices.index.year > start) & (df_cp_indices.index.year < end)]
    
    if show_plot : 
        fig = px.line(df_cp_indices.set_index(df_cp_indices.index.astype("str")), 
            title= ("normalized capacity factor and climate indices, smoothing={}".format(smoothing_duration) if smoothing else "normalized capacity factor and climate indices"))
        fig.update_traces(visible='legendonly')
        fig.update_traces(visible=True, selector=dict(name="capacity_factor"))
        fig.show()
        
    
    return df_cp_indices.corr().capacity_factor[:-1]
 
df_cp = pd.read_csv("dataset_with_timestamp")
df_cp = df_cp.drop(['hour', 'month', 'year', 'day'], axis=1)
df_cp['timestamp'] = pd.to_datetime(df_cp['timestamp'], format='%Y-%m-%d %H:%M:%S')

df_cp = df_cp.groupby(["timestamp","country"]).mean().reset_index()
df_cp = df_cp.set_axis(df_cp.timestamp)
df_cp = df_cp.drop(["timestamp"], axis=1)



indices = ["nao", "ao","mjo80e","mjo40w","mjo20e","mjo160e","mjo10w","nino34"]

df_indices = pd.read_csv("daily_indices_82_to_19.csv")
df_indices['timestamp'] = pd.to_datetime(df_indices['timestamp'], format='%Y-%m-%d')
df_indices = df_indices[["timestamp"]+indices]
df_indices = df_indices.set_axis(df_indices.timestamp)
df_indices = df_indices.drop(["timestamp"], axis=1)

In [ ]:
data_resolution = "d"
selected_country = ["GB"]
normalize_cp=True
smoothing = False
smoothing_duration=6
normalization = "mean"

df_cp_indices = get_cp_indices_df(df_cp, df_indices, selected_coutry=selected_country, data_resolution=data_resolution, normalization=normalization,normalize_cp=normalize_cp , smoothing=smoothing, smoothing_duration=smoothing_duration)

In [ ]:
max_lag = 100
indices = ['nao','ao','mjo80e',	'mjo40w',	'mjo20e',	'mjo160e'	,'mjo10w',	'nino34']
offsets = range(-max_lag,max_lag)



rs = pd.DataFrame(np.array([[crosscorr(df_cp_indices.capacity_factor,df_cp_indices[index], lag) for lag in offsets] for index in indices]).T, columns = indices, index=list(offsets))

fig = px.line(rs).update_layout(xaxis_title="shift (in days)", yaxis_title="correlation daily capacity_factor vs daily climate index")
fig.add_vline(0, annotation_text="center", annotation_position = "top left")
# fig.add_vline(offsets[np.argmax(rs)], annotation_text='Peak synchrony', annotation_position = "top right")
fig.update_traces(visible='legendonly')
fig.show()
fig.write_html("TLCC_daily_cf_GB.html")





